<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/04_Tokenizers_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizers (PyTorch)


Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

分词器是 NLP pipeline的核心组件之一。它们服务于一个目的：将文本转换为模型可以处理的数据。模型只能处理数字，因此分词器需要将我们的文本输入转换为数字数据。在本节中，我们将探索标记化管道中到底发生了什么。

在 NLP 任务中，一般处理的数据都是原始文本。然而，模型只能处理数字，所以我们需要找到一种方法将原始文本转换为数字。这就是标记器所做的，并且有很多方法可以解决这个问题。目标是找到最有意义的表示——即对模型最有意义的表示——如果可能的话，找到最小的表示。

让我们看一下标记化算法的一些示例，并尝试回答您可能对标记化提出的一些问题。

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Word-based基于单词

想到的第一种分词器是基于单词的。通常只需几条规则就可以很容易地设置和使用，而且通常会产生不错的结果。例如，在下图中，目标是将原始文本拆分为单词并为每个单词找到一个数字表示：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg)

有多种拆分文本的方法。例如，我们可以通过应用 Python 的 split() 函数，使用空格将文本标记为单词：

In [ ]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


还有一些词分词器的变体，它们有额外的标点符号规则。使用这种分词器，我们最终可以得到一些相当大的“词汇表”，其中词汇表由我们在语料库中拥有的独立标记的总数定义。

每个单词都会分配一个 ID，从 0 开始，一直到词汇表的大小。该模型使用这些 ID 来识别每个单词。

如果我们想用基于单词的分词器完全覆盖一种语言，我们需要为该语言中的每个单词都有一个标识符，这将生成大量的分词。例如，英语中有超过 500,000 个单词，因此要构建从每个单词到输入 ID 的映射，我们需要跟踪那么多的 ID。此外，像“dog”这样的词与“dogs”这样的词的表示方式不同，模型最初无法知道“dog”和“dogs”是相似的：它将这两个词识别为不相关。这同样适用于其他类似的词，如“run”和“running”，模型最初不会认为它们相似。

最后，我们需要一个自定义标记来表示不在我们词汇表中的单词。这被称为“未知”令牌，通常表示为“[UNK]”或“”。如果您看到分词器生成了很多这样的分词，这通常是一个不好的迹象，因为它无法检索一个词的合理表示，并且您在这个过程中正在丢失信息。构建词汇表的目的是让分词器将尽可能少的词分词为未知词。

减少未知标记数量的一种方法是更深入一层，使用**基于字符的标记器**。

## Character-based 基于字符

基于字符的标记器将文本拆分为字符，而不是单词。这有两个主要好处：

* 词汇量要小得多。
* 词汇外（未知）标记要少得多，因为每个单词都可以从字符构建。

但这里也出现了一些关于空格和标点符号的问题：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg)

这种方法也不完美。由于现在的表示是基于字符而不是单词，人们可能会说，从直觉上讲，它的意义不大：每个字符本身并不意味着很多，而单词就是这种情况。但是，这又因语言而异。例如，在中文中，每个字符比拉丁语中的一个字符包含更多信息。

另一件需要考虑的事情是，我们最终会得到大量的标记要由我们的模型处理：而一个单词对于基于单词的分词器来说只是一个标记，它可以很容易地变成 10 个或更多标记转换为字符时。

为了两全其美，我们可以使用结合这两种方法的第三种技术：子词标记化。

## Subword tokenization 子词分词

子词标记化算法依赖于频繁使用的词不应该被拆分成更小的子词的原则，但是不常见的词应该被分解成有意义的子词。

例如，“annoyingly”可能被认为是一个稀有词，可以分解为“annoyingly”和“ly”。这些都可能更频繁地作为独立子词出现，而同时“annoying”和“ly”的复合含义保留了“annoyingly”的含义。

下面是一个示例，展示了子词标记化算法如何标记序列“让我们进行标记化！”：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg)

这些子词最终提供了很多语义：例如，在上面的示例中，“tokenization”被拆分为“token”和“ization”，这两个标记具有语义意义的同时具有空间效率（只需要两个标记代表一个长词）。这使得我们能够使用较小的词汇表获得相对较好的覆盖率，并且几乎没有位置标记。

这种方法在诸如土耳其语的凝集语言中特别有用，在这种语言中，您可以通过将子词串在一起来形成（几乎）任意长的复杂单词。

## And more!

还有更多的技术。举几个例子：

* GPT-2 中使用的字节级 BPE
* WordPiece，用于 BERT
* SentencePiece 或 Unigram，用于多种多语言模型

## Loading and saving 加载和保存

加载和保存标记器与模型一样简单。实际上，它基于相同的两个方法： from_pretrained() 和 save_pretrained() 。这些方法将加载或保存分词器使用的算法（有点像模型的架构）及其词汇表（有点像模型的权重）。

加载使用与 BERT 相同的检查点训练的 BERT 分词器与加载模型的方式相同，我们使用 BertTokenizer 类：

In [ ]:
from  transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained("bert-base-cased")

与 AutoModel 类似， AutoTokenizer 类将根据检查点名称在库中获取适当的分词器类，并且可以直接与任何检查点一起使用：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

保存分词器与保存模型相同：

In [ ]:
tokenizer.save_pretrained("directory_on_my_computer")

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

我们将在第 3 章详细讨论 token_type_ids ，稍后我们将解释 attention_mask 键。首先，让我们看看 input_ids 是如何生成的。为此，我们需要查看分词器的中间方法。

## Encoding 编码

将文本转换为数字称为编码。编码分两步完成：标记化，然后转换为输入 ID。

正如我们所见，第一步是将文本拆分为单词（或单词的一部分、标点符号等），通常称为标记。有多个规则可以管理该过程，这就是为什么我们需要使用模型名称实例化分词器，以确保我们使用与预训练模型时相同的规则。

第二步是将这些标记转换成数字，这样我们就可以用它们构建一个张量并将它们提供给模型。为此，tokenizer 有一个词汇表，这是我们使用 from_pretrained() 方法实例化它时下载的部分。同样，我们需要使用与预训练模型时相同的词汇表。

为了更好地理解这两个步骤，我们将分别探讨它们。请注意，我们将使用一些方法分别执行部分标记化管道，以向您展示这些步骤的中间结果，但实际上，您应该直接在输入上调用标记器（如第 2 节所示）。

## Tokenization

分词过程由分词器的 tokenize() 方法完成,此方法的输出是字符串列表或标记：

In [ ]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")

sequence="Using a Transformer network is simple"
tokens=tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


这个分词器是一个子词分词器：它拆分单词直到它获得可以用其词汇表表示的标记。 transformer 就是这种情况，它被分成两个标记： transform 和 ##er 。

## From tokens to input IDs

到输入 ID 的转换由 convert_tokens_to_ids() tokenizer 方法处理：

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


这些输出一旦转换为适当的框架张量，就可以用作本章前面所见模型的输入。

In [ ]:
decoded_string = tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014])
print(decoded_string)

Using a Transformer network is simple


请注意，decode方法不仅将索引转换回标记，而且还将属于同一单词的标记组合在一起以生成可读的句子。当我们使用预测新闻本的模型（从提示生成的文本，或者用于翻译或摘要等序列到序列问题）时，这种行为将非常有用。

到目前为止，您应该了解标记器可以处理的原子操作：标记化、转换为 ID 以及将 ID 转换回字符串。然而，我们只是触及了冰山一角。在下一节中，我们将把我们的方法带到它的极限并看看如何克服它们。

In [ ]:
from transformers import AutoTokenizer

checkpoint="bert-base-chinese"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

sequences=[
    "我的双北永不BE！",
    "后来我才明白，有主见，是对我而言最高的评价。"
]
tokens=tokenizer.tokenize(sequences)

print(tokens)

['我', '的', '双', '北', '永', '不', '[UNK]', '！', '后', '来', '我', '才', '明', '白', '，', '有', '主', '见', '，', '是', '对', '我', '而', '言', '最', '高', '的', '评', '价', '。']
